In [ ]:
from bs4 import BeautifulSoup
import requests

# Scraping data to use

In [ ]:
with open('data.csv', "w", encoding='utf-8') as file:
    file.write('"CompanyName";"Location";"Datum";"Price";"Rocket";"StatusRocket";"StatusMission"\n')

In [ ]:
web = 'https://nextspaceflight.com/launches/past/?page=1&search='
while True:
    links = []
    html = requests.get(web)
    soup = BeautifulSoup(html.content, 'html.parser')
    button = soup.find('div', {'style': 'float: right; margin-right: 5px'})

    entries = soup.find_all('div', 'mdl-card__actions mdl-card--border')
    entries = [entry.a for entry in entries]
    for entry in entries:
        links.append(entry['href']) # every mission on this page

    for link in links:
        mission = requests.get('https://nextspaceflight.com'+link)
        
        company_name = None
        location = None
        datum = None
        rocket_status = None
        mission_status = None
        price = None
        rocket_name = None
        soup = BeautifulSoup(mission.content, 'html.parser')
        mission_status = soup.find('h6', {'class': 'rcorners status'})
        if mission_status:
            mission_status = mission_status.text.strip()
        
        rocket_name = soup.find('h4', {'class': 'mdl-card__title-text'})
        if rocket_name:
            rocket_name = rocket_name.text.strip()
        
        details = soup.find_all('div', {'class': 'mdl-cell mdl-cell--6-col-desktop mdl-cell--12-col-tablet'})
        
        datum = soup.find('span', {'id': 'localized'})
        
        details = [detail.text.strip() for detail in details]
        if details:
            company_name = details.pop(0)
        
        for detail in details:
            if 'Price' in detail:
                price = detail.split(':')[1]
            elif 'Status' in detail:
                rocket_status = detail.split(':')[1]
    
        location = soup.find_all('section', {'class': 'card section--center white mdl-grid mdl-grid--no-spacing mdl-shadow--6dp'})
        if location:
            location = location[1].h4.text
        if datum:
            datum = datum.text.strip()
        scraped = f'{str(company_name)};{str(location)};{str(datum)};{str(price)};{str(rocket_name)};{str(rocket_status)};{str(mission_status)}\n'
        with open('data.csv', "a", encoding='utf-8') as file:
            file.write(scraped)
    
    if button:
        web = 'https://nextspaceflight.com/launches/past/' + button.a['href']
        print(web)
    else:
        break